In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jbcse\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Jbcse\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jbcse\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data = pd.read_csv('comments.csv', sep=',', header=0)
data.head()

data[data['Is Reply'] == True][1300:1310]

,Comments,Comment ID,Reply Count,Is Reply,Author,AuthorID
2537,👆👆👆\nThanks for watching and leaving a comment...,Ugw2NYiIjCiRnr5mUpx4AaABAg.9aMePefk-wu9aMh9GrQhE0,0,True,Telegram@👉👉mrbeast770,UCwgMHGG8IDcYjjSXtYbE9rQ
2538,Congratulations you have been selected amongst...,Ugw2NYiIjCiRnr5mUpx4AaABAg.9aMePefk-wu9aMefMoxJJh,0,True,TELEGRAM 👉 MRBEAST6688,UCOqxwnL0H8oCTHlBNO9uSGQ
2540,👆👆👆\nThanks for watching and leaving a comment...,UgzjDdzgbDNfewCcgDt4AaABAg.9aMeN7_EuE49aMhIHnRt_M,0,True,Telegram@👉👉mrbeast770,UCwgMHGG8IDcYjjSXtYbE9rQ
2542,Congratulations you have been selected amongst...,UgyTmxKtlsBG3GDOW1x4AaABAg.9aMdm3Zkln99aMeGjHL_3I,0,True,TELEGRAM 👉 MRBEAST6688,UCOqxwnL0H8oCTHlBNO9uSGQ
2544,👆👆👆\nThanks for watching and leaving a comment...,UgyXisXjRZrctz-CbRB4AaABAg.9aMdLQlECGh9aMhacrkx89,0,True,Telegram@👉👉mrbeast770,UCwgMHGG8IDcYjjSXtYbE9rQ
2546,👆👆👆\nThanks for watching and leaving a comment...,Ugwqas8o6Pyp3EsSYI54AaABAg.9aMcvfgRaUA9aMhjlZooGS,0,True,Telegram@👉👉mrbeast770,UCwgMHGG8IDcYjjSXtYbE9rQ
2547,Congratulations you have been selected amongst...,Ugwqas8o6Pyp3EsSYI54AaABAg.9aMcvfgRaUA9aMd4F67uBs,0,True,TELEGRAM 👉 MRBEAST6688,UCOqxwnL0H8oCTHlBNO9uSGQ
2549,Congratulations you have been selected amongst...,UgyvVKPc9cIMrSF0gXh4AaABAg.9aMc_lWesUB9aMcg3T2VBB,0,True,TELEGRAM 👉 MRBEAST6688,UCOqxwnL0H8oCTHlBNO9uSGQ
2551,Inbox me to claim yours,UgwPBW2kYkJFMl9Mled4AaABAg.9aMcL7oR_0s9aMdFBjyt4n,0,True,TELEGRAM 👉 MRBEAST6688,UCOqxwnL0H8oCTHlBNO9uSGQ
2552,Are you the real mr beast,UgwPBW2kYkJFMl9Mled4AaABAg.9aMcL7oR_0s9aMcxRAs7DY,0,True,Yasanda Ransilu,UCMfTLkkdd0yFLVxkI1Dcxjw


In [3]:
scamChannels = ['UCwgMHGG8IDcYjjSXtYbE9rQ', 'UCIAYvaVP15Cel4NZ1ib_ADA', 'UC8Oy99fOvCjHfbvTImDXpkg',
'UCswOElw6g7pEeAA5Mu15p3Q', 'UCIknJ8HTOMLSfIAmtsm84vA', 'UCsXIMkerN0ofYQVIvNWr7Dg', 'UC5iFMKp-Tuf2RX8wTDIA00w',
'UC4BTXtDeOzz85XMShFWY1VA', 'UCIIab0_13sTLxQY66QjpM-g', 'UCZIIrgNdsq0MPdbG6utOLmw', 'UCf3mY9fz0oesuFS4fUnhjsg',
'UCii92DZYgGqYquT71D9cWSQ', 'UCSkoxrUobYERf5FI1F0KDTg', 'UCbdM2ysSVcPxHghp50tiPQw', 'UC1Wi-CaZ_u111EET3es-jzA',
'UCJB7ZvEbo-xRZum_3Zmq9sw', 'UCe5DcGeti6adyFedf49MRuA', 'UCfOYoX3Cwcn0A8t32c3cM5g', 'UCIAYvaVP15Cel4NZ1ib_ADA',
'UC8Oy99fOvCjHfbvTImDXpkg', 'UCgvlSYolCHq5JiiosIvM6lw', 'UCcxxhVKa9wVvHXscwMXC2gQ', 'UCs5NZ-lIbR_rvYDaKvBrVFw',
'UCHCtAgNRSrcBMRqU5SfHplw', 'UC07gLBHCOrcOyE0bw8Lv8Jg', 'UCnOW6JnwbE46hKOPyDYxc_w', 'UCLlW4UJkWDqW9Ht4u8LRztw',
'UCoPBjpeflrKudIDsvDEV0aw', 'UCcoKvgQgWLXosFP5giUeI4g', 'UChYGnlCGDagZT0F8GYhDY2w', 'UCapA77PmpW9dEZQxxJY7TIg',
'UCfdoVOJ9krvIZReKoLWAelA', 'UCU5WWthBfCSYPt-YRNRZYPQ', 'UCDDbjiG_3PHqn4_8BRMr9yA', 'UCVnjpK8HoaeYGQCJUjtiq6g']


def isScam(id):

    if  id in scamChannels:
        return 1
    else:
        return 0


data['Is Scam'] = data['AuthorID'].apply(lambda x: isScam(x))

train = data[0:30000].copy()[['Comments', 'Is Reply', 'Author', 'Is Scam']]
test = data[30001:35001].copy()[['Comments', 'Is Reply', 'Author']]

In [4]:
x = train['Is Scam'].value_counts()
x

0    25821
1     4179
Name: Is Scam, dtype: int64

In [ ]:
train['Word Count'] = train['Comments'].apply(lambda x: len(str(x).split()))
print(train[train['Is Scam']==1]['Word Count'].mean())
print(train[train['Is Scam']==0]['Word Count'].mean())

In [ ]:
# PLOTTING WORD-COUNT
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,4))
train_words=train[train['Is Scam']==1]['Word Count']
ax1.hist(train_words,color='red')
ax1.set_title('Scam Comments')
train_words=train[train['Is Scam']==0]['Word Count']
ax2.hist(train_words,color='green')
ax2.set_title('Non-scam Comments')
fig.suptitle('Words per comment')
plt.show()

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = str(text).lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
train['Clean Comment'] = train['Comments'].apply(lambda x: finalpreprocess(x))

In [ ]:
train.head()

In [ ]:
train['Clean text tokens']=[nltk.word_tokenize(i) for i in train['Clean Comment']]
model = Word2Vec(train['Clean text tokens'],min_count=1)     
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

X_train, X_test, y_train, y_test = train_test_split(train['Clean Comment'], train['Is Scam'], test_size=0.4, shuffle=True)

X_train_tok = [nltk.word_tokenize(i) for i in X_train]
X_test_tok = [nltk.word_tokenize(i) for i in X_test]

In [ ]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])



modelw = MeanEmbeddingVectorizer(w2v)

# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_test_vectors_w2v = modelw.transform(X_test_tok)

In [ ]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  #model

y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]

print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
#Pre-processing the new dataset
test['Clean Comments'] = test['Comments'].apply(lambda x: finalpreprocess(x)) #preprocess the data
X_test=test['Clean Comments']
#converting words to numerical data using tf-idf
X_vector=tfidf_vectorizer.transform(X_test)
#use the best model to predict 'target' value for the new dataset 
y_predict = lr_tfidf.predict(X_vector)      
y_prob = lr_tfidf.predict_proba(X_vector)[:,1]

test['predict_prob']= y_prob
test['Is Scam']= y_predict
final=test[['Clean Comments', 'predict_prob', 'Is Scam']].reset_index(drop=True)

In [ ]:
final